In [78]:
import json
import requests
import os
from fuzzywuzzy import fuzz
import numpy as np

API_KEY = os.getenv('GOOGLE_MAP_API_KEY')

In [4]:
hotels_yelp = []

# Open the file and read it line by line
with open('../Datasets/Hotels.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        hotels_yelp.append(json_obj)


In [5]:
hotels_google = []


# Open the file and read it line by line
with open('../Datasets/Hotels_Google_Candidates.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        hotels_google.append(json_obj)

In [12]:
list_yelp = []
for hotely in hotels_yelp:
    list_yelp.append(hotely['name'])

In [20]:
list_google = []
for hotelg in hotels_google:
    candidates = hotelg['candidates']
    cands = []
    for candidate in candidates: 
            cands.append(candidate['name'])
    list_google.append(cands)

In [21]:
for i in range(len(list_google)):
    print(list_yelp[i],list_google[i])

Aloft Philadelphia Airport ['Aloft Philadelphia Airport']


The Inn at The Union League ['Inn At The Union League']


North American Motor Inns ['Philly Inn & Suites']


Kimpton Hotel Monaco Philadelphia ['Kimpton Hotel Monaco Philadelphia']


DoubleTree by Hilton Hotel Philadelphia Center City ['DoubleTree by Hilton Hotel Philadelphia Center City']


Roosevelt Inn ['Roosevelt Inn']


Philadelphia Marriott Old City []


Rodeway Inn Center City ['Quality Inn - Center City', 'Rodeway Inn Center City']


Element Philadelphia ['Element Philadelphia Downtown']


W Philadelphia ['W Philadelphia']


Loews Philadelphia Hotel ['Loews Philadelphia Hotel']


The Windsor Suites ['The Windsor Suites']


The Logan Philadelphia, Curio Collection by Hilton ['The Logan Philadelphia, Curio Collection by Hilton']


Live Casino Hotel ['Live! Casino & Hotel Philadelphia']


Hampton Inn Philadelphia-International Airport ['Hampton Inn Philadelphia-International Airport']


Motto By Hilton Philadelphia Ritte

In [22]:
#North American Motor Inns ['Philly Inn & Suites'] -- they change name
#Philadelphia Marriott Old City [] -- address is sepereated by comma
#Rodeway Inn Center City ['Quality Inn - Center City', 'Rodeway Inn Center City'] -- the rodeway is closed, and now it's called quality inn
#Blue Cross RiverRink Winterfest ['Independence Blue Cross RiverRink'] -- name changed
#Peking Inn [] -- They moved already, to a new place in 4906
#Hawthorn Suites By Wyndham Philadelphia Airport ['Philadelphia Suites Extended Stay Hotel'] -- they change name, or the one from yelp is closed
#Penrose Hotel ['Holiday Inn Philadelphia Arpt-Stadium Area, an IHG Hotel'] -- permanently closed
#Southern Inn ['Simply Good Food', 'SOUTH Restaurant & Jazz Club'] -- permanently closed
#Tokio Sushi Bistro & Bed & Breakfast ['M Sushi'] -- none of them shows up on google 
#Courtyard by Marriott Philadelphia Airport ['Sonesta Select Philadelphia Airport'] -- change into new name?

In [37]:
#Philadelphia Marriott Old City []
#this is a hotel that shown on google properly, why we get empty.
name = 'Pekin Inn'
address = f'One Dock St, 2nd and Walnut St'
city = 'Philadelphia'

query = f'{name},{address}, {city}'

url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=formatted_address%2Cname%2Cplace_id&input={query}&inputtype=textquery&key={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error for {name}:", response.status_code)

{'candidates': [{'formatted_address': '4905 Catharine St, Philadelphia, PA 19143, United States', 'name': 'Peking Inn', 'place_id': 'ChIJeR4kHevGxokRsQbUNAClJcw'}], 'status': 'OK'}


In [48]:
#try to use phrase similarity to determine if the business is still the same or not
print(list_yelp[1],", ", list_google[1][0])
similarity_score = fuzz.ratio(list_yelp[1].lower(), list_google[1][0].lower())
print("similarity_score: ",similarity_score)

The Inn at The Union League ,  Inn At The Union League
similarity_score:  92


In [66]:
def find_similar_words(word1, word2):
    similarity_score = fuzz.ratio(word1.lower(), word2.lower())
    return similarity_score

In [67]:
scores = []
for i in range(len(list_google)):
    word1 = list_yelp[i]
    if(len(list_google[i])):
        word2 = list_google[i][0]
    else:
        word2 = ''
    score = find_similar_words(word1, word2)
    scores.append(score)
scores

[100,
 92,
 32,
 100,
 100,
 100,
 0,
 75,
 82,
 100,
 100,
 100,
 100,
 68,
 100,
 100,
 51,
 72,
 100,
 85,
 65,
 75,
 100,
 78,
 100,
 100,
 100,
 100,
 100,
 100,
 62,
 86,
 100,
 0,
 100,
 100,
 88,
 100,
 100,
 98,
 93,
 88,
 100,
 85,
 100,
 100,
 100,
 100,
 100,
 71,
 96,
 100,
 37,
 100,
 100,
 100,
 100,
 100,
 99,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 47,
 100,
 85,
 100,
 100,
 100,
 100,
 26,
 21,
 100,
 73,
 100,
 100,
 87,
 85,
 100,
 55,
 96,
 58,
 100,
 100,
 28,
 100,
 100,
 100,
 100,
 47,
 64,
 100,
 60,
 84,
 100,
 80,
 100,
 100,
 39,
 77,
 67,
 100]

In [69]:
from Levenshtein import ratio


def find_similar_words_ratio(word1, word2):
    similarity_ratio = ratio(word1.lower(), word2.lower())
    return similarity_ratio

In [71]:
scores_ratio = []
for i in range(len(list_google)):
    word1 = list_yelp[i]
    if(len(list_google[i])):
        word2 = list_google[i][0]
    else:
        word2 = ''
    score = find_similar_words(word1, word2)
    scores_ratio.append(score)
scores_ratio

#same thing, just speed difference

[100,
 92,
 32,
 100,
 100,
 100,
 0,
 75,
 82,
 100,
 100,
 100,
 100,
 68,
 100,
 100,
 51,
 72,
 100,
 85,
 65,
 75,
 100,
 78,
 100,
 100,
 100,
 100,
 100,
 100,
 62,
 86,
 100,
 0,
 100,
 100,
 88,
 100,
 100,
 98,
 93,
 88,
 100,
 85,
 100,
 100,
 100,
 100,
 100,
 71,
 96,
 100,
 37,
 100,
 100,
 100,
 100,
 100,
 99,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 47,
 100,
 85,
 100,
 100,
 100,
 100,
 26,
 21,
 100,
 73,
 100,
 100,
 87,
 85,
 100,
 55,
 96,
 58,
 100,
 100,
 28,
 100,
 100,
 100,
 100,
 47,
 64,
 100,
 60,
 84,
 100,
 80,
 100,
 100,
 39,
 77,
 67,
 100]

In [74]:
count_below_60 = sum(1 for x in scores if x < 60)
count_below_60

13

In [82]:
indices = np.where(np.array(scores) < 60)
indices

(array([  2,   6,  16,  33,  52,  68,  75,  76,  84,  86,  89,  94, 103]),)

In [83]:
count_below_60 = sum(1 for x in scores if x < 50)
count_below_60

10

Our final decision is keep all the old one, and add the new ones into the dataset

we can use other API to get the reviews if we want

Clean up the candidates formatting

In [84]:
hotels_google = []


# Open the file and read it line by line
with open('../Datasets/Hotels_Google_Candidates.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        hotels_google.append(json_obj)

In [89]:
hotels_google[6]['candidates'][0]

IndexError: list index out of range

In [90]:
hotels_google_clean = []
for hotel_google in hotels_google:
    candidates = hotel_google['candidates']
    if(len(candidates)):
        hotels_google_clean.append(candidates[0])
    else:
        hotels_google_clean.append({})

#write to jsonl file
with open('../Datasets/Hotels_Google_Candidates_clean.jsonl', 'w') as file:
    for hotel_google in hotels_google_clean:
        json.dump(hotel_google, file)
        file.write('\n')